In [1]:
import geopandas as gpd
from usda.utils import AttrDict

__C=AttrDict() 
args=__C

__C.gi=AttrDict()
__C.gi.epsg_wgs84=4326
__C.gi.Chicago_epsg=32616

__C.data=AttrDict()
__C.data.chicago_boundary_fn=r'G:\data\Chicago_boundaries_city\Chicago_boundaries_city.shp'

__C.data.naip_root=r'F:\data\NAIP_chicago'
__C.data.naip_mosaic_fn=r'F:\data\naip_chicago_mosaic.tif'

In [2]:
import pathlib
import rioxarray
import pystac_client
import planetary_computer
from shapely.geometry import shape
from IPython.display import Image
import stackstac

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [8]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

boundary=gpd.read_file(args.data.chicago_boundary_fn)

area_of_interest = {
    "type": "Polygon",
    "coordinates": [list(boundary.geometry[0].envelope.exterior.coords)],
    }

range_date="2015-01-01/2016-01-01"
search_naip=catalog.search(collections=["naip"], intersects=area_of_interest, datetime=range_date)
items_naip=search_naip.item_collection()
print(f"{len(items_naip)} Items found in the date range")
items_naip

50 Items found in the date range


id: il_m_4108720_se_16_1_20150824_20151021
"bbox: [-87.566678, 41.621978, -87.495873, 41.690511]"
gsd: 1.0
datetime: 2015-08-24T00:00:00Z
naip:year: 2015
"proj:bbox: [452842.0, 4607961.0, 458691.0, 4615534.0]"
proj:epsg: 26916
naip:state: il
"proj:shape: [7573, 5849]"
"proj:transform: [1.0, 0.0, 452842.0, 0.0, -1.0, 4615534.0, 0.0, 0.0, 1.0]"
https://stac-extensions.github.io/eo/v1.0.0/schema.json


In [9]:
area_shape = shape(area_of_interest)
target_area = area_shape.area

def area_of_overlap(item):
    overlap_area = shape(item.geometry).intersection(shape(area_of_interest)).area
    return overlap_area / target_area

items_naip_=sorted(items_naip, key=area_of_overlap, reverse=True)
print(len(items_naip_))
items_naip_[:3]

50


[<Item id=il_m_4108702_sw_16_1_20150822_20151021>,
 <Item id=il_m_4108702_nw_16_1_20150822_20151021>,
 <Item id=il_m_4108710_nw_16_1_20150822_20151021>]

In [5]:
Image(url=items_naip_[0].assets["rendered_preview"].href)

In [10]:
ds_0=rioxarray.open_rasterio(items_naip_[0].assets["image"].href)# .sel(band=[1, 2, 3])
ds_0

<xarray.DataArray (band: 4, y: 7593, x: 5854)>
[177797688 values with dtype=uint8]
Coordinates:
  * band         (band) int32 1 2 3 4
  * x            (x) float64 4.271e+05 4.271e+05 ... 4.329e+05 4.329e+05
  * y            (y) float64 4.644e+06 4.644e+06 ... 4.636e+06 4.636e+06
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [7]:
ds=stackstac.stack(items_naip_)
ds

<xarray.DataArray 'stackstac-725bf33222b1dc249c250fda1e22655e' (time: 50,
                                                                band: 1,
                                                                y: 49539,
                                                                x: 42374)>
dask.array<fetch_raster_window, shape=(50, 1, 49539, 42374), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * time        (time) datetime64[ns] 2015-08-22 2015-08-22 ... 2015-08-24
    id          (time) <U38 'il_m_4108702_sw_16_1_20150822_20151021' ... 'il_...
  * band        (band) <U5 'image'
  * x           (x) float64 4.164e+05 4.164e+05 ... 4.588e+05 4.588e+05
  * y           (y) float64 4.658e+06 4.657e+06 ... 4.608e+06 4.608e+06
    gsd         float64 1.0
    naip:state  <U2 'il'
    proj:epsg   int32 26916
    naip:year   <U4 '2015'
    title       <U14 'RGBIR COG tile'
    epsg        int32 26916
Attributes:
    spec:        RasterSpec(epsg=26916, bounds=(416397.0, 4607961.0, 458771.0...
    crs:         epsg:26916
    transform:   | 1.00, 0.00, 416397.00|\n| 0.00,-1.00, 4657500.00|\n| 0.00,...
    resolution:  1.0

In [9]:
naip_url_lst=[item.assets['image'].href.split('?')[0] for item in items_naip_]
from torchvision.datasets.utils import download_url

failed_download_info={}
for idx,item_url in enumerate(naip_url_lst):
    print(idx,end=" ")
    try:
        download_url(item_url,args.data.naip_root)
    except:
        failed_download_info[idx]=item_url

0 Downloading https://naipeuwest.blob.core.windows.net/naip/v002/il/2015/il_100cm_2015/41087/m_4108702_sw_16_1_20150822.tif to F:\data\NAIP_chicago\m_4108702_sw_16_1_20150822.tif


100%|█████████████████████████████████████████████████████| 186139876/186139876 [01:31<00:00, 2028283.34it/s]


1 Downloading https://naipeuwest.blob.core.windows.net/naip/v002/il/2015/il_100cm_2015/41087/m_4108702_nw_16_1_20150822.tif to F:\data\NAIP_chicago\m_4108702_nw_16_1_20150822.tif


100%|█████████████████████████████████████████████████████| 186908973/186908973 [01:00<00:00, 3096779.52it/s]


2 Downloading https://naipeuwest.blob.core.windows.net/naip/v002/il/2015/il_100cm_2015/41087/m_4108710_nw_16_1_20150822.tif to F:\data\NAIP_chicago\m_4108710_nw_16_1_20150822.tif


...

30 Downloading https://naipeuwest.blob.core.windows.net/naip/v002/il/2015/il_100cm_2015/41087/m_4108718_se_16_1_20150822.tif to F:\data\NAIP_chicago\m_4108718_se_16_1_20150822.tif


100%|█████████████████████████████████████████████████████| 183994329/183994329 [01:33<00:00, 1958515.99it/s]


31 Downloading https://naipeuwest.blob.core.windows.net/naip/v002/il/2015/il_100cm_2015/41087/m_4108719_sw_16_1_20150822.tif to F:\data\NAIP_chicago\m_4108719_sw_16_1_20150822.tif


 64%|██████████████████████████████████▋                   | 118751232/184675672 [09:56<05:31, 198995.93it/s]


32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

In [4]:
import usda.geodata_process as usda_geoprocess

usda_geoprocess.raster_mosaic(args.data.naip_root,args.data.naip_mosaic_fn)

RasterioIOError: 'F:/data/NAIP_chicago/m_4108701_nw_16_1_20150822.tif' not recognized as a supported file format.

In [3]:
from torchvision.datasets.utils import download_url

download_url(r'https://naipeuwest.blob.core.windows.net/naip/v002/il/2015/il_100cm_2015/41087/m_4108719_sw_16_1_20150822.tif',args.data.naip_root)

  0%|          | 0/184675672 [00:00<?, ?it/s]

In [4]:
import torch
torch.cuda.is_available()

False

In [5]:
torch.cuda.device_count()

0

In [8]:
torch.__version__

'1.13.1+cpu'

In [1]:
from fastai.vision.all import *
from fastai.text.all import *
from fastai.collab import *
from fastai.tabular.all import *

In [2]:
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

C:\Users\richi\anaconda3\envs\gpd\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\richi\anaconda3\envs\gpd\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\richi/.cache\torch\hub\checkpoints\resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.168194,0.022756,0.008796,12:30


epoch,train_loss,valid_loss,error_rate,time
0,0.045253,0.024095,0.006766,17:06


In [3]:
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(224))

learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.163845,0.017815,0.005413,12:44


epoch,train_loss,valid_loss,error_rate,time
0,0.062094,0.025217,0.008796,16:47
